# Real Multi-Agent Pipeline: Research → Writer → Editor

This notebook demonstrates a **real production multi-agent pipeline** using LangChain with CERT SDK instrumentation.

## Pipeline Architecture

```
User Query → Researcher Agent → Writer Agent → Editor Agent → Final Output
```

**Agents:**
1. **Researcher**: Gathers information and key points
2. **Writer**: Creates initial content draft
3. **Editor**: Refines and polishes the final output

**CERT Metrics Measured:**
- Individual agent quality scores
- Context propagation effect (γ) - performance changes from accumulated context
- Pipeline health score
- Execution timing and observability

**Estimated time:** 3-5 minutes

## Setup and Installation

In [ ]:
# Install required packages
# !pip install cert-sdk langchain langchain-openai langchain-core

In [ ]:
import cert
from cert.integrations.langchain import CERTLangChain

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

## Enter API Key

In [ ]:
from getpass import getpass

api_key = getpass("Enter your OpenAI API key: ")

## Initialize CERT Provider and Integration

In [ ]:
# Create CERT provider for baseline comparison
cert_provider = cert.create_provider(
    api_key=api_key,
    model_name="gpt-4o",
    temperature=0.7,
    max_tokens=1024,
)

# Get validated baseline
baseline = cert.ModelRegistry.get_model("gpt-4o")

print(f"✓ Using {baseline.model_id}")
print(f"  Baseline: C={baseline.consistency:.3f}, μ={baseline.mean_performance:.3f}")

In [ ]:
# Initialize CERT LangChain integration
cert_integration = CERTLangChain(
    provider=cert_provider,
    baseline=baseline,
    verbose=True,  # Print execution details
)

print("✓ CERT LangChain integration initialized")

## Define the Three Agents

We'll create three specialized agents using **LangChain's LCEL (LangChain Expression Language)**.
This is the modern, recommended approach - much simpler than custom classes!

In [ ]:
# Initialize LangChain LLM
llm = ChatOpenAI(
    api_key=api_key,
    model="gpt-4o",
    temperature=0.7,
)

print("✓ LangChain LLM initialized")

In [ ]:
# Agent 1: Researcher
# Uses LCEL pattern: prompt | llm
researcher_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a research expert. Analyze the user's question and provide key research points, facts, and insights. Be thorough and factual."),
    ("human", "{input}"),
])

# Simple wrapper class to make LCEL chains work with CERT's message format
class MessageAdapter:
    """Adapts LangChain LCEL chains to work with CERT's message format."""
    def __init__(self, chain):
        self.chain = chain
    
    def invoke(self, input_data):
        # Extract input from messages format
        messages = input_data.get("messages", [])
        if messages:
            last_msg = messages[-1]
            user_input = last_msg.get("content", "") if isinstance(last_msg, dict) else str(last_msg)
        else:
            user_input = input_data.get("input", "")
        
        # Run the chain
        response = self.chain.invoke({"input": user_input})
        
        # Return in messages format
        return {
            "messages": [
                {"role": "user", "content": user_input},
                {"role": "assistant", "content": response.content}
            ]
        }

# Create researcher using LCEL
researcher_chain = researcher_prompt | llm
researcher = MessageAdapter(researcher_chain)

print("✓ Researcher agent created (using LangChain LCEL)")

In [ ]:
# Agent 2: Writer
writer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional writer. Take the research points provided and create a well-structured, engaging article. Focus on clarity and flow."),
    ("human", "Research points: {input}"),
])

writer_chain = writer_prompt | llm
writer = MessageAdapter(writer_chain)

print("✓ Writer agent created (using LangChain LCEL)")

In [ ]:
# Agent 3: Editor
editor_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert editor. Review the draft article and improve it by fixing grammar, enhancing clarity, and ensuring professional quality. Keep the core content but polish it."),
    ("human", "Draft to edit: {input}"),
])

editor_chain = editor_prompt | llm
editor = MessageAdapter(editor_chain)

print("✓ Editor agent created (using LangChain LCEL)")

## Create Instrumented Pipeline

**Important:** `create_multi_agent_pipeline()` automatically wraps agents with CERT instrumentation.
You don't need to call `wrap_agent()` separately!

In [ ]:
# Create the pipeline - wrapping happens automatically inside this function
pipeline = cert_integration.create_multi_agent_pipeline([
    {"agent": researcher, "agent_id": "researcher", "agent_name": "Research Agent"},
    {"agent": writer, "agent_id": "writer", "agent_name": "Writer Agent"},
    {"agent": editor, "agent_id": "editor", "agent_name": "Editor Agent"},
])

print("✓ Multi-agent pipeline created with CERT instrumentation")
print("\n  Pipeline: Research → Write → Edit → Final Output")
print("\n  Note: All agents are automatically wrapped with CERT metrics")

## Run the Pipeline

Let's test the pipeline with a real query.

In [ ]:
# Define the user query
user_query = "Explain the key factors in building successful multi-model LLM systems for production."

print(f"User Query: {user_query}")
print("\n" + "="*70)
print("Executing Pipeline...")
print("="*70)

In [ ]:
# Run the pipeline
result = pipeline({"messages": [{"role": "user", "content": user_query}]})

print("\n" + "="*70)
print("Pipeline Execution Complete")
print("="*70)

## View Results

In [ ]:
# Display final output
final_output = result["messages"][-1]["content"]

print("\n" + "="*70)
print("FINAL OUTPUT")
print("="*70)
print(final_output)
print("="*70)

## View CERT Metrics

Now let's see the CERT metrics collected during execution.

In [ ]:
# Print comprehensive metrics
cert_integration.print_metrics()

## Interpret the Results

### Quality Scores
- Each agent's output is scored for semantic relevance, linguistic coherence, and content density
- Higher scores (closer to 1.0) indicate better quality

### Context Propagation Effect (γ)
**What it measures:**
- Performance changes when models process accumulated context
- How attention mechanisms handle extended context in sequential processing

**What it does NOT measure:**
- ❌ Agent coordination, collaboration, or planning
- ❌ Intelligence or reasoning capabilities
- ❌ WHY context helps (black box measurement)

**Interpretation:**
- **γ > 1.0**: Sequential context accumulation improves performance
- **γ = 1.0**: No benefit from accumulated context
- **γ < 1.0**: Context accumulation degrades performance

### Pipeline Health
- **H > 0.8**: Production ready - deploy with confidence
- **0.6 < H < 0.8**: Acceptable - deploy with monitoring
- **H < 0.6**: Needs investigation before production

### Execution Timing
- Shows duration for each agent
- Helps identify bottlenecks

## Get Metrics Programmatically

In [ ]:
# Access metrics as dictionary
metrics_dict = cert_integration.get_metrics_summary()

print("Metrics Summary:")
for key, value in metrics_dict.items():
    print(f"  {key}: {value}")

## Alternative: Use Wrapped Agents Directly

If you don't want to use `create_multi_agent_pipeline()`, you can wrap and execute agents manually:

In [ ]:
# Alternative approach: Wrap agents manually
# (Only run this if you didn't use create_multi_agent_pipeline above)

# Reset metrics
cert_integration.reset_metrics()

# Wrap each agent individually
instrumented_researcher = cert_integration.wrap_agent(
    agent=researcher,
    agent_id="researcher",
    agent_name="Research Agent",
    calculate_quality=True,
)

instrumented_writer = cert_integration.wrap_agent(
    agent=writer,
    agent_id="writer",
    agent_name="Writer Agent",
    calculate_quality=True,
)

instrumented_editor = cert_integration.wrap_agent(
    agent=editor,
    agent_id="editor",
    agent_name="Editor Agent",
    calculate_quality=True,
)

# Execute pipeline manually
result = {"messages": [{"role": "user", "content": user_query}]}
result = instrumented_researcher.invoke(result)
result = instrumented_writer.invoke(result)
result = instrumented_editor.invoke(result)

# Calculate metrics manually
cert_integration.calculate_coordination_effect()
cert_integration.calculate_pipeline_health()

print("✓ Manual pipeline execution complete")
cert_integration.print_metrics()

## Try Different Queries

Test the pipeline with different types of queries to see how context propagation effects vary.

In [ ]:
# Example queries to try
example_queries = [
    "What are the challenges in deploying LLM agents to production?",
    "Explain how to measure AI agent performance and reliability.",
    "Compare different multi-model frameworks for enterprise applications.",
]

print("Try these queries:")
for i, query in enumerate(example_queries, 1):
    print(f"  {i}. {query}")

## Production Deployment

### Key Takeaways:

1. **Use CERT metrics** to validate your pipeline before production
2. **Monitor context propagation effect (γ)** - if γ drops, investigate sequential processing behavior
3. **Track health score** - set alerts if it falls below your threshold
4. **Measure consistently** - run CERT measurements regularly to detect drift

### What CERT Measures:

✅ **Engineering Characterization:**
- Statistical behavior of sequential LLM processing
- Performance changes from context accumulation
- Attention mechanism effects (black box measurement)
- Operational metrics for deployment decisions

❌ **What CERT Does NOT Measure:**
- Agent coordination or collaboration
- Intelligence or reasoning capabilities
- WHY models improve with context

This is **engineering instrumentation**, not coordination science.

### Next Steps:

- Try the CrewAI integration: `examples/crewai_pipeline.ipynb`
- Learn about custom baselines: `examples/advanced_usage.py`
- Explore the full API: See README.md for documentation